# Data Loading

In [1]:
import pandas as pd
import numpy as np
!pip install lightgbm
!pip install xgboost
import xgboost as xgb
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import auc, classification_report, roc_auc_score
from sklearn.model_selection import StratifiedKFold

!pip install catboost
from catboost import CatBoostClassifier, Pool

In [2]:
train= pd.read_csv('/content/drive/MyDrive/Women/Train.csv')
test= pd.read_csv('/content/drive/MyDrive/Women/Test.csv')
ss= pd.read_csv('/content/drive/MyDrive/Women/SampleSubmission.csv')
vd= pd.read_csv('/content/drive/MyDrive/Women/VariableDefinitions.csv')

In [3]:
train.head()

,ID,gender,age,race,dwelling,dwelling_type,province_code,metro_code,psu,nationality,RTH,marital_st,Lang_inside,Lang_outside,Education,lw_work,lw_business,help_on_household,job_or_business,nature_of_work,target
0,ID_00008683,Female,20,Black African,Dwelling/house or brick/concrete block structu...,Formal,North West,NW - Non Metro,66710095,South Africa,Son/daughter/stepchild/adopted child of person 01,Single and have never been married/never lived...,Xitsonga,Setswana,Grade 11/Standard 9/Form 4,No,No,No,No,Not applicable,0
1,ID_00061163,Female,42,Black African,Dwelling/house or brick/concrete block structu...,Formal,Limpopo,LP - Non Metro,98510279,South Africa,Other relative (e.g. in-laws or aunt/uncle) of...,Living together like husband and wife,Sepedi,Sepedi,Grade 12/Standard 10/Form 5/Matric (No Exemption),Yes,No,No,Not applicable,Permanent,0
2,ID_00071403,Female,86,Black African,Dwelling/house or brick/concrete block structu...,Formal,Eastern Cape,EC - Non Metro,29210321,South Africa,Father/mother/stepfather/stepmother of person 01,Widowed,IsiXhosa,IsiXhosa,Unspecified,No,No,No,No,Not applicable,0
3,ID_00077803,Female,20,Black African,Town house (semi-detached house in complex),Formal,KwaZulu-Natal,KZN - eThekwini,59913860,South Africa,Son/daughter/stepchild/adopted child of person 01,Single and have never been married/never lived...,IsiZulu,IsiZulu,Grade 12/Standard 10/Form 5/Matric (No Exemption),No,No,No,No,Not applicable,0
4,ID_00086763,Female,26,Black African,Town house (semi-detached house in complex),Formal,Gauteng,GP - City of Johannesburg,79813828,South Africa,Son/daughter/stepchild/adopted child of person 01,Single and have never been married/never lived...,Setswana,English,Grade 12/Standard 10/Form 5/Matric (No Exemption),No,No,No,No,Not applicable,0


In [4]:
test.head()

,ID,gender,age,race,dwelling,dwelling_type,province_code,metro_code,psu,nationality,RTH,marital_st,Lang_inside,Lang_outside,Education,lw_work,lw_business,help_on_household,job_or_business,nature_of_work
0,ID_00241643,Female,23,Coloured,Dwelling/house or brick/concrete block structu...,Formal,Northern Cape,NC - Non Metro,36410104,South Africa,Head/acting head,Single and have never been married/never lived...,Afrikaans,Afrikaans,Grade 10/Standard 8/Form 3,Yes,No,No,Not applicable,Permanent
1,ID_00312043,Female,64,Black African,Dwelling/house or brick/concrete block structu...,Formal,Limpopo,LP - Non Metro,97410477,South Africa,Head/acting head,Widowed,Afrikaans,Afrikaans,Grade 12/Standard 10/Form 5/Matric (No Exemption),No,No,No,No,Not applicable
2,ID_00345323,Female,18,Black African,Unspecified,Unspecified,Eastern Cape,EC - Buffalo City,26011572,South Africa,Son/daughter/stepchild/adopted child of person 01,Single and have never been married/never lived...,IsiXhosa,IsiXhosa,Grade 11/Standard 9/Form 4,No,No,No,No,Not applicable
3,ID_00369643,Female,19,Black African,Dwelling/house or brick/concrete block structu...,Formal,Mpumalanga,MP - Non Metro,87710674,South Africa,Son/daughter/stepchild/adopted child of person 01,Single and have never been married/never lived...,Xitsonga,Sepedi,Grade 12/Standard 10/Form 5/Matric (No Exemption),No,No,No,No,Not applicable
4,ID_00381163,Female,24,Black African,Dwelling/house or brick/concrete block structu...,Formal,Mpumalanga,MP - Non Metro,87411106,South Africa,Head/acting head,Single and have never been married/never lived...,SiSwati,SiSwati,Grade 11/Standard 9/Form 4,No,No,No,No,Not applicable


In [5]:
train.dtypes

ID                   object
gender               object
age                   int64
race                 object
dwelling             object
dwelling_type        object
province_code        object
metro_code           object
psu                   int64
nationality          object
RTH                  object
marital_st           object
Lang_inside          object
Lang_outside         object
Education            object
lw_work              object
lw_business          object
help_on_household    object
job_or_business      object
nature_of_work       object
target                int64
dtype: object

In [6]:
train.isnull().sum()

ID                   0
gender               0
age                  0
race                 0
dwelling             0
dwelling_type        0
province_code        0
metro_code           0
psu                  0
nationality          0
RTH                  0
marital_st           0
Lang_inside          0
Lang_outside         0
Education            0
lw_work              0
lw_business          0
help_on_household    0
job_or_business      0
nature_of_work       0
target               0
dtype: int64

# Data Encoding

In [7]:
cols= ['ID', 'gender', 'race', 'dwelling', 'dwelling_type', 'province_code', 'metro_code', 'nationality', 'RTH', 'marital_st', 'Lang_inside', 'Lang_outside', 'Education', 'lw_work',
       'lw_business', 'help_on_household', 'job_or_business', 'nature_of_work']

In [8]:
for y in cols:
  le= LabelEncoder()
  test[y]= le.fit_transform(test[y].values)

In [9]:
for y in cols:
  le= LabelEncoder()
  train[y]= le.fit_transform(train[y].values)

# Modeling

In [10]:
X= train.drop('target', axis=1)

In [11]:
def seed_everything(seed):
    np.random.seed(seed)
SEED = 25  
seed_everything(SEED)

In [12]:
y= train['target']

In [13]:
#Parameters
lgb_params = {'boosting_type': 'gbdt','objective': 'binary','metric': 'auc', 'learning_rate': 0.005,
              'n_estimators': 5000,'early_stopping_rounds': 300,'sub_sample' : 0.5,'max_depth':17, 'reg_alpha':1,'reg_lambda':2,'subsample_freq':1,
              'colsample_bytree' : 0.2,'seed': SEED,'silent':False
            }

In [14]:

score = 0
test['target'] = 0
skf = StratifiedKFold(n_splits=15,shuffle=True, random_state=SEED)
oof_lgb = np.zeros((train.shape[0],))
lgb_preds = []
for fold_, (trn_idx, val_idx) in enumerate(skf.split(X, y)):
    print('Fold:',fold_+1)
        
    # Creating lgb train/valid data
    tr_x, tr_y = X.iloc[trn_idx,:], y[trn_idx] 
    vl_x, vl_y = X.iloc[val_idx,:], y[val_idx] 
        
    train_data = lgb.Dataset(tr_x, label=tr_y,categorical_feature=cols)
    valid_data = lgb.Dataset(vl_x, label=vl_y,categorical_feature=cols)
    # Train Model
    
    estimator = lgb.train(
                          lgb_params,
                          train_data,
                          valid_sets = [train_data,valid_data],
                          verbose_eval = 100,
                        )

    y_pred_val = estimator.predict(vl_x,num_iteration=estimator.best_iteration)
    oof_lgb[val_idx] = y_pred_val
    score = score + roc_auc_score(vl_y, y_pred_val)
    y_pred_test = estimator.predict(test,num_iteration=estimator.best_iteration)
    lgb_preds.append(y_pred_test)

print('OOF score :',roc_auc_score(y, oof_lgb))

Fold: 1
Training until validation scores don't improve for 300 rounds.


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:741: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings

[100]	training's auc: 0.743871	valid_1's auc: 0.653046
[200]	training's auc: 0.750482	valid_1's auc: 0.661187
[300]	training's auc: 0.757366	valid_1's auc: 0.661134
[400]	training's auc: 0.764998	valid_1's auc: 0.659821
[500]	training's auc: 0.771367	valid_1's auc: 0.661082
[600]	training's auc: 0.776377	valid_1's auc: 0.660242
Early stopping, best iteration is:
[319]	training's auc: 0.758914	valid_1's auc: 0.663655
Fold: 2
Training until validation scores don't improve for 300 rounds.


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:741: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings

[100]	training's auc: 0.746413	valid_1's auc: 0.597899
[200]	training's auc: 0.751847	valid_1's auc: 0.5927
[300]	training's auc: 0.758118	valid_1's auc: 0.595063
Early stopping, best iteration is:
[1]	training's auc: 0.689094	valid_1's auc: 0.655777
Fold: 3
Training until validation scores don't improve for 300 rounds.


...

[100]	training's auc: 0.748696	valid_1's auc: 0.634368
[200]	training's auc: 0.753107	valid_1's auc: 0.632579
[300]	training's auc: 0.759269	valid_1's auc: 0.629526
Early stopping, best iteration is:
[99]	training's auc: 0.74843	valid_1's auc: 0.634684
Fold: 14
Training until validation scores don't improve for 300 rounds.


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:741: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings

[100]	training's auc: 0.741215	valid_1's auc: 0.642526
[200]	training's auc: 0.748739	valid_1's auc: 0.638053
[300]	training's auc: 0.754941	valid_1's auc: 0.642737
Early stopping, best iteration is:
[8]	training's auc: 0.744715	valid_1's auc: 0.645632
Fold: 15
Training until validation scores don't improve for 300 rounds.


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:741: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings

[100]	training's auc: 0.743869	valid_1's auc: 0.627605
[200]	training's auc: 0.748934	valid_1's auc: 0.631579
[300]	training's auc: 0.75505	valid_1's auc: 0.636789
Early stopping, best iteration is:
[11]	training's auc: 0.741095	valid_1's auc: 0.646737
OOF score : 0.6528866513453655


In [15]:
lightgbm_preds = np.mean(lgb_preds,axis=0)
lightgbm_preds

array([0.08010346, 0.06535318, 0.07357316, ..., 0.08051291, 0.09515759,
       0.12804304])

In [16]:
ss.target= lightgbm_preds
ss.to_csv('Umoja', index=False)